
# Forecast

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization, Conv1D, MaxPooling1D, Concatenate, \
                                    LSTM, Activation, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MSE
from tensorflow_addons.layers import SpectralNormalization as SN
from tqdm import trange

In [ ]:
%matplotlib inline
file_name = 'attack_regressor'
dir_name = file_name + '_dir'
weights_dir = 'forecast_dir/ckpt'
os.mkdir(dir_name)

In [ ]:
data = np.load('../../data/processed/normalized_generated_data.npz')
gen_electrical_data = data['electrical_data']
gen_labels = data['labels']

In [ ]:
data = np.load('../../../../data/processed/full_data.npz')
real_electrical_data = data['electrical_data'][:500]
real_labels = data['labels'][:500]

In [ ]:
signal = np.load('../../../../data/processed/labeled_data.npz')['electrical_values']
difs = np.max(signal, axis=1) - np.min(signal, axis=1)
miu = np.mean(difs)
sigma = np.std(difs)
print('miu =', miu)
print('sigma =', sigma)

## Split Data

In [ ]:
def split_data(data, labels):
    train_factor, val_factor = 0.8, 0.1
    train_limit, val_limit = int(len(data) * train_factor), int(len(data) * (train_factor + val_factor))
    train_data, train_labels = data[:train_limit], labels[:train_limit]
    val_data, val_labels = data[train_limit:val_limit], labels[train_limit:val_limit]
    test_data, test_labels = data[val_limit:], labels[val_limit:]
    return train_data, train_labels, val_data, val_labels, test_data, test_labels

## Building the Model

In [ ]:
data_length = 720 # One month of data
nr_labels = 6 # from labels + timestamp
emb_input_dim = 28 + gen_electrical_data.shape[1] - data_length + 1 # Added values are for timestamp
emb_output_dim = 32
nr_samples = 500
batch_size = nr_samples
untargeted_vals = {}
targeted_max_vals = {}
targeted_min_vals = {}
interval = np.linspace(0.01, 0.1, 10)
plot_interval = np.linspace(0, 0.1, 11)
attack_names = ['FGSM', 'BIM', 'PGD']
sample_index = 7

In [ ]:
def conv_block(kernel_size, filters, padding, input_layer):
    sn_conv_1d = SN(Conv1D(kernel_size=kernel_size, filters=filters, padding=padding))(input_layer)
    bn = BatchNormalization()(sn_conv_1d)
    act = Activation('relu')(bn)
    mp_1d = MaxPooling1D()(act)
    res = Dropout(0.2)(mp_1d)
    return res

In [ ]:
def create_model():
    sampled_data = Input((data_length, gen_electrical_data.shape[2]), name='input_1')
    sampled_labels = Input((nr_labels,), name='input_2')
    label_embedding = SN(Embedding(emb_input_dim, emb_output_dim, input_length=nr_labels))(sampled_labels)
    flatten_embedding = Flatten()(label_embedding)
    dense_embedding = SN(Dense(64))(flatten_embedding)
    conv1 = conv_block((3), 128, 'same', sampled_data)
    conv2 = conv_block((5), 128, 'same', sampled_data)
    conv3 = conv_block((7), 128, 'same', sampled_data)
    conc_convs = Concatenate(axis=-1)([conv1, conv2, conv3])
    lstm1 = LSTM(128, return_sequences=True)(conc_convs)
    bn1 = BatchNormalization()(lstm1)
    lstm2 = LSTM(128)(bn1)
    bn2 = BatchNormalization()(lstm2)
    conc_all = Concatenate(axis=1)([bn2, dense_embedding])
    dense_final = SN(Dense(128, activation='relu'))(conc_all)
    res = SN(Dense(1, activation='tanh'))(dense_final)
    model = Model([sampled_data, sampled_labels], res, name='C-RNN')
    model.compile(optimizer = Adam(1e-3, 0.5, 0.9), loss = 'mse', metrics = ['mae', 'mape'])
    return model

In [ ]:
real_model = create_model()
real_model.load_weights(weights_dir + '/weights.ckpt')
gen_model = create_model()
gen_model.load_weights(weights_dir + '/gen_weights.ckpt')

## Create Data

In [ ]:
def generate_data(data, labels, nr_samples=500):
    indexes = np.random.randint(0, data.shape[0], nr_samples)
    sampled_data = data[indexes]
    hours = np.random.randint(0, data.shape[1] - data_length, nr_samples)
    X = np.array([x[h:h + data_length] for x, h in zip(sampled_data, hours)])
    l = np.array([np.concatenate((l, h), axis=-1) for l, h in zip(labels[indexes], np.expand_dims(hours, -1))])
    y = np.array([x[h + data_length, 0] for x, h in zip(sampled_data, hours)])
    return X, l, y

## Define Attacks

### FGSM

In [ ]:
def FGSM(X, l, y, model, eps, targeted, clip_min=-1, clip_max=1):
    with tf.GradientTape() as tape:
        tape.watch(X)
        y_pred = model([X, l])
        loss = MSE(y, y_pred)
        if targeted:
            loss = -loss
    grad = tape.gradient(loss, X)
    normalized_grad = tf.sign(grad)
    normalized_grad = tf.stop_gradient(normalized_grad)
    scaled_grad = eps * normalized_grad
    X_adv = X + scaled_grad
    X_adv = tf.clip_by_value(X_adv, clip_min, clip_max)
    return X_adv

### PGD

In [ ]:
def PGD(X, l, y, model, eps, targeted, eps_iter=None, max_iter=10, rand_init=True, clip_min=-1, clip_max=1):
    if eps_iter == None:
        eps_iter = eps / 3
    if rand_init:
        eta = tf.random.uniform(tf.shape(X), -eps, eps)
    else:
        eta = tf.zeros_like(X)
    eta = tf.clip_by_value(eta, -eps, eps)
    X_adv = X + eta
    for i in trange(max_iter):
        X_adv = FGSM(X_adv, l, y, model, eps_iter, targeted, clip_min, clip_max)
        eta = X_adv - X
        eta = tf.clip_by_value(eta, -eps, eps)
        X_adv = X + eta
        X_adv = tf.clip_by_value(X_adv, clip_min, clip_max)
    return X_adv

### BIM

In [ ]:
def BIM(X, l, y, model, eps, targeted, eps_iter=None, max_iter=10, clip_min=-1, clip_max=1):
    return PGD(X, l, y, model, eps, targeted, eps_iter, max_iter, False, clip_min, clip_max)

## Attacks

In [ ]:
max_targets = tf.ones((nr_samples), name='max_targets')
min_targets = -tf.ones((nr_samples), name='min_targets')

for electrical_data, labels, model in [(real_electrical_data, real_labels, real_model), \
                                       (gen_electrical_data, gen_labels, gen_model)]:
    _, _, _, _, test_data, test_labels = split_data(electrical_data, labels)
    X, l, y = generate_data(test_data, test_labels, nr_samples)
    X_tf = tf.convert_to_tensor(X, dtype=tf.float32, name='X')
    l_tf = tf.convert_to_tensor(l, dtype=tf.int32, name='l')
    y_tf = tf.convert_to_tensor(y, dtype=tf.float32, name='y')
    print('Original results:')
    mse, mae, mape = model.evaluate([X, l], y, batch_size=batch_size)
    clean_result = (mse, mae, mape, 0., X[sample_index])
    clean_results = np.array([clean_result for _ in plot_interval], dtype=object)
    untargeted_vals['Orig'] = clean_results
    targeted_max_vals['Orig'] = clean_results
    targeted_min_vals['Orig'] = clean_results
    for attack_name in attack_names:
        print('For', attack_name, 'attack:')
        attack_type = eval(attack_name)
        for results_dict, targeted, y_tar in [(untargeted_vals, False, y_tf), (targeted_max_vals, True, max_targets), \
                                      (targeted_min_vals, True, min_targets)]:
            vals = [clean_result]
            for eps in interval:
                print('For eps = %.2f:' % eps)
                X_adv = attack_type(X_tf, l_tf, y_tar, model, eps, targeted)
                mse, mae, mape = model.evaluate([X_adv, l], y_tar, batch_size=batch_size)
                perturbation = np.nanmean(np.abs((X_adv - X)))
                vals.append((mse, mae, mape, perturbation, X_adv[sample_index]))
            results_dict[attack_name] = np.array(vals, dtype=object)

## Plot results

In [ ]:
result_types = ['No Target', 'Max Target', 'Min Target']
colors = ['g', 'b', 'r', 'c']
linestyles=['-','-','-.','--']
for i, metric in enumerate(['MSE', 'MAE', 'MAPE', 'Perturbation']):
    fig, axs = plt.subplots(1, 3)
    fig.set_size_inches(30, 10)
    fig.suptitle(metric)
    for j, results_dict in enumerate([untargeted_vals, targeted_max_vals, targeted_min_vals]):
        ax = axs[j]
        ax.title.set_text(result_types[j])
        ax.set_xlabel('epsilon')
        ax.set_ylabel('Values')
        for k, attack in enumerate(['Orig'] + attack_names):
            ax.plot(plot_interval, results_dict[attack][:, i], color=colors[k], linestyle=linestyles[k], label=attack)
        ax.legend()

In [ ]:
untargeted_vals_2 = {'untargeted_' + key: val for key, val in untargeted_vals.items()}
targeted_max_vals_2 = {'targeted_max_' + key: val for key, val in targeted_max_vals.items()}
targeted_min_vals_2 = {'targeted_min_' + key: val for key, val in targeted_min_vals.items()}
np.savez_compressed(dir_name + '/attack_regressor_metrics.npz', **untargeted_vals_2, **targeted_max_vals_2, **targeted_min_vals_2)